In [3]:
from datetime import datetime
from typing import Optional, Sequence, Annotated
import operator


from pydantic import BaseModel, Field


from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, get_buffer_string, BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command



# Optional: checkpointer for testing
from langgraph.checkpoint.memory import InMemorySaver

In [4]:
class MessagesState(BaseModel):
    messages: Annotated[Sequence[BaseMessage], operator.add] = []

class AgentInputState(MessagesState):
    user_id: Optional[str] = None
    topic: Optional[str] = None

class AgentState(MessagesState):
    research_brief: Optional[str] = None
    supervisor_messages: Annotated[Sequence[BaseMessage], operator.add] = []
    raw_notes: Annotated[list[str], operator.add] = []
    notes: Annotated[list[str], operator.add] = []
    final_report: Optional[str] = None

class ClarifyWithUser(BaseModel):
    need_clarification: bool = Field(description="Whether clarification is needed")
    question: str = Field(description="Question to ask the user for clarification")
    verification: str = Field(description="Verification message confirming understanding")


class ResearchQuestion(BaseModel):
    research_brief: str = Field(description="Comprehensive research brief")

In [5]:
clarify_with_user_instructions = '''
You are a research assistant helping to clarify research requests.
Given the conversation history, determine if you need more information.


Conversation History:
{messages}


Today's date: {date}


If the request is clear enough to begin research, respond with:
- need_clarification: false
- verification: A brief confirmation of what you'll research
- question: "N/A"


If you need more information, respond with:
- need_clarification: true
- verification: "N/A"
- question: A specific question to clarify the research needs
'''


transform_messages_into_research_topic_prompt = '''
Based on the conversation history, create a comprehensive research brief.
The brief should include:
- Clear research objectives
- Key questions to answer
- Scope and limitations
- Any specific requirements mentioned


Conversation History:
{messages}


Today's date: {date}
'''

In [6]:
def get_today_str() -> str:
    return datetime.now().strftime("%a %b %d, %Y")


from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0
)


In [7]:
from typing import Literal

def clarify_with_user(state: AgentState) -> Command[Literal["write_research_brief", "__end__"]]:
    clarified = False
    while not clarified:
        # Create a simple instruction prompt
        prompt = f"""You are a helpful research assistant.Conversation so far:{get_buffer_string(state.messages)}
                    If the user's request is not clear, ask one clarification question.
                    Otherwise, say 'Understood. Proceeding with research.'"""
        
        response = model.invoke([HumanMessage(content=prompt)])
        ai_message = response.content.strip()
        print(f"\nAI: {ai_message}")

        # If AI says “Understood…”, move to next phase
        if "understood" in ai_message.lower() or "proceeding" in ai_message.lower():
            clarified = True
            state.messages.append(AIMessage(content=ai_message))
        else:
            # Ask user for clarification
            user_input = input("You: ")
            state.messages.append(AIMessage(content=ai_message))
            state.messages.append(HumanMessage(content=user_input))
    return Command(goto="write_research_brief", update={"messages": state.messages})


def write_research_brief(state: AgentState):
    prompt = f"""You are an expert research analyst.Conversation so far:{get_buffer_string(state.messages)}
                Write a clear, structured research brief including:
                1. Research Objectives
                2. Key Questions
                3. Scope and Limitations
                4. Specific Requirements"""
    response = model.invoke([HumanMessage(content=prompt)])
    research_brief = response.content.strip()

    print(f"\nAI Research Brief:\n\n{research_brief}\n")
    return {
        "research_brief": research_brief,
        "supervisor_messages": [HumanMessage(content=f"{research_brief}.")]
    }

In [8]:
deep_researcher_builder = StateGraph(AgentState, input_schema=AgentInputState)
deep_researcher_builder.add_node("clarify_with_user", clarify_with_user)
deep_researcher_builder.add_node("write_research_brief", write_research_brief)
deep_researcher_builder.add_edge(START, "clarify_with_user")
deep_researcher_builder.add_edge("write_research_brief", END)

checkpointer = InMemorySaver()
scope_research = deep_researcher_builder.compile(checkpointer=checkpointer)

In [13]:
from langchain_core.messages import HumanMessage, AIMessage

print("AI Research Assistant Initialized")
state = {"messages": [HumanMessage(content="Let's start research.")]}

while True:
    result = scope_research.invoke(state, config={"configurable": {"thread_id": "1"}})
    print(f"\nAI: {result['messages'][-1].content}")

    if "research_brief" in result:
        print("\n Research process completed.")
        print(result["research_brief"])
        break

    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Conversation ended.")
        break

    state["messages"].append(HumanMessage(content=user_input))


AI Research Assistant Initialized

AI: What topic or subject would you like to research?

AI: Understood. Proceeding with research.

AI Research Brief:

**Research Brief: Latest News in India**

**1. Research Objectives:**
*   To provide a concise and up-to-date overview of the most significant news developments and headlines in India.
*   To identify key events, trends, and topics currently dominating the Indian national discourse across various sectors.

**2. Key Questions:**
*   What are the top 3-5 most prominent national news headlines in India as of the most recent available data?
*   Are there any major political developments (e.g., policy changes, elections, government announcements)?
*   What are the latest updates on the Indian economy (e.g., market trends, inflation, major business news)?
*   Are there any significant social issues, cultural events, or environmental concerns being widely reported?
*   Has India been involved in any notable international relations news or glo